In [208]:
import os
import re
from dataclasses import dataclass
from typing import List
import shutil

import torch.nn as nn
import torch
import pandas as pd
import torchvision
from PIL import Image
import torchvision.transforms as transforms
import numpy as np

## Make student directories

In [214]:
exclude = [
    "arceatom","ballspencer","bhargavaaman","carnaffanadam","castillejosyviel",
    "chengbenjamin","chenjacky","chiamkatarina","chudavid","cyphuskieran",
]


if True:

    student_names = pd.value_counts([x.split('_')[0] for x in os.listdir('./submissions/')]).keys()
    
    for student in student_names:
        if student in exclude:
            continue

        if not os.path.isdir('./student_submissions/{}'.format(student)):
            os.makedirs('./student_submissions/{}'.format(student))

        shutil.copyfile(
            './ECE324_A4P2_rubric.xlsx', 
            './student_submissions/{}/ECE324_A4P2_rubric.xlsx'.format(student)
        )

        student_files = [x for x in os.listdir('./submissions/') if x.startswith(student)]

        for student_file in student_files:
            shutil.copyfile('./submissions/{}'.format(student_file), './student_submissions/{}/{}'.format(
                student,
                student_file
            ))


### Construct data class

In [117]:
@dataclass
class StudentModel:
    model_name: str
    student_name: str
    model_size: str
    model_perf: float
        
    
student_models = []

for model_file in model_files:
    student_name = re.split('_|\.', model_file)[0]
    model_size = re.split('_|\.', model_file)[-2].lower()
    
    student_models.append(
        StudentModel(
            model_file,
            student_name,
            'small' if 'small' in model_size else 'big',
            -1)
    )

print(
    "Student model sizes: \n",
    pd.value_counts([x.model_size for x in student_models])
)
print('\n')
print(
    "Number of models per strudent: \n",
    pd.value_counts(pd.value_counts([x.student_name for x in student_models]).values)
)

Student model sizes: 
 small    52
big      51
dtype: int64


Number of models per strudent: 
 2    51
1     1
dtype: int64


In [126]:
model_types = []

for student_model in student_models:
    model = torch.load('./submissions/{}'.format(student_model.model_name), map_location=torch.device('cpu'))
    model_types.append(type(model))
    
pd.value_counts(model_types)

<class 'collections.OrderedDict'>    103
dtype: int64

## Construct Test dataset

In [178]:
transform = transforms.Compose(
    [transforms.ToTensor()]
)

train_dataset = torchvision.datasets.ImageFolder('../ece324_assignment_4_1/training_v2', transform=transform)
train_data = torch.utils.data.DataLoader(train_dataset)

training_stds = []
training_means = []

for image, label in train_data:
    training_means.append(image.mean(axis=(0, 2, 3)).numpy())
    training_stds.append(image.std(axis=(0, 2, 3)).numpy())
    
training_means = np.array(training_means).mean(axis=0)
training_stds = np.array(training_stds).mean(axis=0)

In [184]:
transform = transforms.Compose(
    [
        transforms.ToTensor(), 
        transforms.Normalize(training_means, training_stds)
    ]
)


test_dataset = torchvision.datasets.ImageFolder(
    '../ece324_assignment_4_1/data/test/',
    transform=transform
)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

for image, label in test_data:
    print(label)
    print(image.shape)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

In [211]:
#create CNN

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3,kernel_num,kernel_size) #conv layer1 (#input channels, #kernels, size, (stride=default=1)
    self.bn1 = nn.BatchNorm2d(kernel_num)
    self.pool1 =  nn.MaxPool2d(2,2)
    self.out_dim = 56
    self.out_dim = int((self.out_dim - kernel_size + 1)/2)

    if (conv_num == 2):
      self.conv2 = nn.Conv2d(kernel_num,kernel_num,kernel_size) 
      self.bn2 = nn.BatchNorm2d(kernel_num)        
      self.pool2 = nn.MaxPool2d(2,2)
      self.out_dim = int((self.out_dim - kernel_size + 1)/2)
    if (conv_num == 4):
      self.conv2 = nn.Conv2d(kernel_num,kernel_num,kernel_size)
      self.bn2 = nn.BatchNorm2d(kernel_num)  
      self.pool2 = nn.MaxPool2d(2,2,padding=1)
      self.conv3 = nn.Conv2d(kernel_num,kernel_num,kernel_size)
      self.bn3 = nn.BatchNorm2d(kernel_num)
      self.pool3 = nn.MaxPool2d(2,2)
      self.conv4 = nn.Conv2d(kernel_num,kernel_num,kernel_size)
      self.bn4 = nn.BatchNorm2d(kernel_num)
      self.pool4 = nn.MaxPool2d(2,2)
      self.out_dim = 56
      for i in range(conv_num):
          self.out_dim = int((self.out_dim - kernel_size + 1)/2) 

    self.fc1 = nn.Linear(kernel_num*self.out_dim*self.out_dim,num_nueron1)  
    self.bnl_1 = nn.BatchNorm1d(num_nueron1)    #batch normal 1D 
    self.fc2 = nn.Linear(num_nueron1,10) 

  # def forward(self,x):   #no bn
  #   x = self.pool1(F.relu(self.conv1(x))) #run x through conv1->b->relu->pool
  #   x = self.pool2(F.relu(self.conv2(x))) # run x through conv2->->relu->pool
  #   x = self.pool3(F.relu(self.conv3(x))) # run x through conv2->bn3->relu->pool
  #   x = self.pool4(F.relu(self.conv4(x))) # run x through conv2->bn4->relu->pool
  #   x = x.view(-1, kernel_num*self.out_dim*self.out_dim) #reshapes tensor in to 1D
  #   x = F.relu(self.fc1(x))
  #   x = self.fc2(x)
  #   return x  

  def forward(self,x):
    x = self.pool1(F.relu(self.bn1(self.conv1(x)))) #run x through conv1->bn1->relu->pool
    x = self.pool2(F.relu(self.bn2(self.conv2(x)))) # run x through conv2->bn2->relu->pool
    x = self.pool3(F.relu(self.bn3(self.conv3(x)))) # run x through conv2->bn3->relu->pool
    x = self.pool4(F.relu(self.bn4(self.conv4(x)))) # run x through conv2->bn4->relu->pool
    x = x.view(-1, kernel_num*self.out_dim*self.out_dim) #reshapes tensor in to 1D
    x = F.relu(self.bnl_1(self.fc1(x)))
    x = self.fc2(x)
    return x

In [212]:
conv_num = 4
kernel_num = 30
num_nueron1 = 8
kernel_size = 3
#hyperparameters

lr = .1
batch_size = 32

num_epoch = 5

loss_type = 1

seed = 1
# N=3 #eval_every

torch.manual_seed(seed)
Net()

Net(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv3): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=10, out_features=8, bias